\begin{cases}
min \ \frac{1}{2} ||W||^2 \\
s.t. \  y_i(W*X+b) \geq 1
\end{cases}

SVM 自带 $L2$ 正则项 $\frac{1}{2}||W||^2$, 正则化手段是抑制过拟合的一种方法，因为发生过拟合时参数$W$一般是较大的值，而正则化主要是对$W$进行惩罚，从而降低模型的复杂性。\\

通过拉格朗日变换得到新的目标函数
$L_P = \frac{1}{2}\ {\left \| W \right \|}^{2} - \sum_{i=1}^{N}\lambda_i(y_i(W  \cdot  X + b) - 1) $

对 $Lp$ 关于 $W$ 和 $b$ 求偏导，并令它们等于零得:

$\frac{\partial L_P}{\partial W} = 0 \Rightarrow W = \sum_{i=1}^{N}\lambda_i y_i X_i$

$\frac{\partial L_P}{\partial b} = 0 \Rightarrow \sum_{i=1}^{N}\lambda_i y_i = 0$

这种变换导致如下拉格朗日乘子约束，称作KTT条件

$\lambda_{i} \geqslant 0$

$\lambda_{i} [y_{i}(W \cdot X_i+b)-1] = 0$

将$W\ b$带入$Lp$得

$L_D = \sum_{i=1}^{N} \lambda_i - \frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N} \lambda_i \lambda_j y_i y_j X_i X_j$


In [1]:
import numpy as np

In [2]:
import cvxopt
import cvxopt.solvers

In [4]:
class Kernel:

    @staticmethod
    def linear():
        return lambda x,y: np.inner(x,y)
    
    @staticmethod
    def gaussian(sigma):
        return lambda X, y: np.exp(-np.sqrt(np.linalg.norm(X-y) ** 2 / (2 * sigma ** 2)))

In [5]:
class SVC:

    def __init__(self, kernel='linear', C=1.0, sigma=1.0, **kwargs):
        if kernel not in ['linear', 'gaussian']:
            raise ValueError("Now only support linear and gaussian kernel")
        elif kernel == 'linear':
            kernel_fn = Kernel.linear()
        else:
            kernel_fn = Kernel.gaussian(sigma)
        self.kernel = kernel_fn  # kernel func
        self.C = C
        self._predictor = None